# Predicting Pitches
For our project, we'll aim to solve the problem of predicting the next pitch a Major League Baseball Pitcher will throw during an at bat

In [1]:
import utils.utils as utils
import pandas as pd

## Part 1: Preprocessing the data

In [2]:
# get the data
pitch_data = utils.get_pitch_data()
pitch_data.head()

,pitch_data_id,season,pitcher_id,team_id,team_abbrev,era,wins,losses,throws,b1_id,...,runs_pitcher_team,runs_batter_team,game_id,inning_id,half_inning_id,at_bat_id,gid,p1_pitch_type,p0_at_bat_o,p0_pitch_des
0,334805,2018,572971,117,HOU,3.57,11.0,10.0,L,572287,...,0,0,e46d949f-eac3-47bb-bd50-2fe05c9feafc,119581e7-5af3-4a4a-b6b9-67f22d760984,41102429-e42f-448e-913e-587b601164c0,5937bd7f-00d6-42b0-bdc5-8745a98309ca,gid_2018_09_19_seamlb_houmlb_1,FF,0,
1,334806,2018,572971,117,HOU,3.57,11.0,10.0,L,572287,...,0,0,e46d949f-eac3-47bb-bd50-2fe05c9feafc,119581e7-5af3-4a4a-b6b9-67f22d760984,41102429-e42f-448e-913e-587b601164c0,5937bd7f-00d6-42b0-bdc5-8745a98309ca,gid_2018_09_19_seamlb_houmlb_1,FF,0,
2,334807,2018,572971,117,HOU,3.57,11.0,10.0,L,572287,...,0,0,e46d949f-eac3-47bb-bd50-2fe05c9feafc,119581e7-5af3-4a4a-b6b9-67f22d760984,41102429-e42f-448e-913e-587b601164c0,5937bd7f-00d6-42b0-bdc5-8745a98309ca,gid_2018_09_19_seamlb_houmlb_1,SL,0,
3,334808,2018,446372,114,CLE,2.02,8.0,2.0,R,456715,...,0,0,0268a063-3acb-498f-b166-019d6080cf3e,5c5faa08-34b4-4570-83cd-0577e4abc0d0,f61b94d3-7da7-4633-a5c4-142806e756de,593958a2-c7f2-4174-a45a-7f9fcb5bd0aa,gid_2018_06_05_milmlb_clemlb_1,SI,0,
4,334809,2018,446372,114,CLE,2.02,8.0,2.0,R,456715,...,0,0,0268a063-3acb-498f-b166-019d6080cf3e,5c5faa08-34b4-4570-83cd-0577e4abc0d0,f61b94d3-7da7-4633-a5c4-142806e756de,593958a2-c7f2-4174-a45a-7f9fcb5bd0aa,gid_2018_06_05_milmlb_clemlb_1,SI,0,


In [3]:
pitch_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 74 columns):
pitch_data_id          10000 non-null int64
season                 10000 non-null int64
pitcher_id             10000 non-null int64
team_id                10000 non-null object
team_abbrev            10000 non-null object
era                    10000 non-null float64
wins                   9958 non-null float64
losses                 9958 non-null float64
throws                 10000 non-null object
b1_id                  10000 non-null int64
b1_team_id             10000 non-null object
b1_stand               10000 non-null object
b1_height              10000 non-null int64
b1_bats                10000 non-null object
b1_avg                 10000 non-null float64
b1_hr                  10000 non-null int64
b1_rbi                 10000 non-null int64
b1_bat_order           9419 non-null float64
b1_game_position       9419 non-null object
p1_pitch_id            10000 non-null objec

In [4]:
cols_to_drop=['p1_pitch_id','p0_pitch_id','result_type','pitch_data_id','season','team_id','game_id','inning_id','half_inning_id','at_bat_id','gid']
# drop unneeded cols
pitch_data = utils.drop_columns_by_list(pitch_data,cols_to_drop)
# drop pickoffs, automatic balls, strikes, etc..
pitch_data = utils.drop_unwanted_pitches(pitch_data)
# set category of cols
pitch_data = utils.set_dtypes(pitch_data)
# drop nans
pitch_data = pitch_data.dropna()
all_data = pitch_data

In [5]:
pitch_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8775 entries, 0 to 9999
Data columns (total 63 columns):
pitcher_id             8775 non-null object
team_abbrev            8775 non-null object
era                    8775 non-null float64
wins                   8775 non-null float64
losses                 8775 non-null float64
throws                 8775 non-null object
b1_id                  8775 non-null object
b1_team_id             8775 non-null object
b1_stand               8775 non-null int64
b1_height              8775 non-null int64
b1_bats                8775 non-null object
b1_avg                 8775 non-null float64
b1_hr                  8775 non-null int64
b1_rbi                 8775 non-null int64
b1_bat_order           8775 non-null float64
b1_game_position       8775 non-null object
p1_pitch_seqno         8775 non-null int64
p0_pitch_seqno         8775 non-null float64
p0_inning              8775 non-null float64
result_type_simple     8775 non-null object
x          

In [6]:
pitch_types = pitch_data.loc[:,'p1_pitch_type']
Y = utils.encode_simple_pitch_types(pitch_types)
X = pitch_data.drop('p1_pitch_type',axis=1)
# one hot encode necessary cols
X = utils.one_hot_encode(X)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8775 entries, 0 to 9999
Columns: 1258 entries, era to p0_pitch_des_
dtypes: float64(35), int64(16), uint8(1207)
memory usage: 13.6 MB


In [7]:
X_train, X_test, y_train, y_test = utils.split_dataset_into_train_and_test(X,Y)

In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7020 entries, 5477 to 8262
Columns: 1258 entries, era to p0_pitch_des_
dtypes: float64(35), int64(16), uint8(1207)
memory usage: 10.9 MB


## Part2: Train an XGBoost multiclassifier

In [20]:
import models.xgboost_model as xgb_model

In [8]:
xgb_multi_class_model = xgb_model.get_multi_class_classifier_model()

In [11]:
xgb_model.fit_multi_class_model(model=xgb_multi_class_model,x_train=X_train,y_train=y_train,x_test=X_test,y_test=y_test,save_location='./training/xgb_multi_class.bin')


Model Report
Accuracy (Train) : 1.0
Accuracy (Test) : 0.4447544642857143


It looks like we're overfitting. I'll remove the pitcher and batter ids

In [21]:
pitch_data = all_data
# remove the pitch
cols_to_drop=['pitcher_id','b1_id']
# drop unneeded cols
pitch_data = utils.drop_columns_by_list(pitch_data,cols_to_drop)
X = pitch_data.drop('p1_pitch_type',axis=1)
# one hot encode necessary cols
X = utils.one_hot_encode(X)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8775 entries, 0 to 9998
Columns: 156 entries, era to p0_pitch_des_
dtypes: float64(35), int64(16), uint8(105)
memory usage: 4.4 MB


In [22]:
X_train, X_test, y_train, y_test = utils.split_dataset_into_train_and_test(X,Y)

In [23]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7020 entries, 5478 to 8314
Columns: 156 entries, era to p0_pitch_des_
dtypes: float64(35), int64(16), uint8(105)
memory usage: 3.5 MB


In [24]:
xgb_multi_class_model = xgb_model.get_multi_class_classifier_model()

In [25]:
xgb_model.fit_multi_class_model(model=xgb_multi_class_model,x_train=X_train,y_train=y_train,x_test=X_test,y_test=y_test,save_location='./training/xgb_multi_class_no_pitcher_batter_id.bin',useTrainCV=True)

/home/jpalomares/.local/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/jpalomares/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)



Model Report
Accuracy (Train) : 0.7210826210826211
Accuracy (Test) : 0.4153846153846154


## Part3: Try an NN

In [16]:
pitch_data = all_data
# remove the pitch
cols_to_drop=['pitcher_id','b1_id']
# drop unneeded cols
pitch_data = utils.drop_columns_by_list(pitch_data,cols_to_drop)
X = pitch_data.drop('p1_pitch_type',axis=1)
# one hot encode necessary cols
X = utils.one_hot_encode(X)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8775 entries, 0 to 9999
Columns: 156 entries, era to p0_pitch_des_
dtypes: float64(35), int64(16), uint8(105)
memory usage: 4.7 MB


In [17]:
import models.nn_model as nn_model
import keras

In [18]:
# 16 pitch types
num_pitch_types = 16
num_cols = len(X.iloc[0,:])
X_train, X_test, y_train, y_test = utils.split_dataset_into_train_and_test(X,Y)
y_test = keras.utils.to_categorical(y_test,num_classes=num_pitch_types)
y_train = keras.utils.to_categorical(y_train,num_classes=num_pitch_types)

In [19]:
model = nn_model.get_multi_class_classifier_model(num_cols,num_pitch_types)

In [20]:
score=nn_model.fit_multi_class_model(model,X_train,y_train,X_test,y_test)

Epoch 1/200
7020/7020 [==============================] - 0s 33us/step - loss: 13.7146 - acc: 0.1457
Epoch 2/200
7020/7020 [==============================] - 0s 12us/step - loss: 12.8279 - acc: 0.2041
Epoch 3/200
7020/7020 [==============================] - 0s 12us/step - loss: 12.8279 - acc: 0.2041
Epoch 4/200
7020/7020 [==============================] - 0s 12us/step - loss: 12.8279 - acc: 0.2041
Epoch 5/200
7020/7020 [==============================] - 0s 12us/step - loss: 12.8279 - acc: 0.2041
Epoch 6/200
7020/7020 [==============================] - 0s 12us/step - loss: 12.8279 - acc: 0.2041
Epoch 7/200
7020/7020 [==============================] - 0s 12us/step - loss: 12.8279 - acc: 0.2041
Epoch 8/200
7020/7020 [==============================] - 0s 12us/step - loss: 12.8279 - acc: 0.2041
Epoch 9/200
7020/7020 [==============================] - 0s 13us/step - loss: 12.8279 - acc: 0.2041
Epoch 10/200
7020/7020 [==============================] - 0s 12us/step - loss: 12.8302 - acc: 0.2040

7020/7020 [==============================] - 0s 13us/step - loss: 12.8279 - acc: 0.2041
Epoch 83/200
7020/7020 [==============================] - 0s 14us/step - loss: 12.8279 - acc: 0.2041
Epoch 84/200
7020/7020 [==============================] - 0s 13us/step - loss: 12.8279 - acc: 0.2041
Epoch 85/200
7020/7020 [==============================] - 0s 12us/step - loss: 12.8279 - acc: 0.2041
Epoch 86/200
7020/7020 [==============================] - 0s 11us/step - loss: 12.8279 - acc: 0.2041
Epoch 87/200
7020/7020 [==============================] - 0s 12us/step - loss: 12.8279 - acc: 0.2041
Epoch 88/200
7020/7020 [==============================] - 0s 12us/step - loss: 12.8279 - acc: 0.2041
Epoch 89/200
7020/7020 [==============================] - 0s 11us/step - loss: 12.8279 - acc: 0.2041
Epoch 90/200
7020/7020 [==============================] - 0s 11us/step - loss: 12.8279 - acc: 0.2041
Epoch 91/200
7020/7020 [==============================] - 0s 12us/step - loss: 12.8279 - acc: 0.2041
Epo

7020/7020 [==============================] - 0s 12us/step - loss: 12.8279 - acc: 0.2041
Epoch 163/200
7020/7020 [==============================] - 0s 11us/step - loss: 12.8279 - acc: 0.2041
Epoch 164/200
7020/7020 [==============================] - 0s 11us/step - loss: 12.8279 - acc: 0.2041
Epoch 165/200
7020/7020 [==============================] - 0s 11us/step - loss: 12.8279 - acc: 0.2041
Epoch 166/200
7020/7020 [==============================] - 0s 11us/step - loss: 12.8279 - acc: 0.2041
Epoch 167/200
7020/7020 [==============================] - 0s 11us/step - loss: 12.8279 - acc: 0.2041
Epoch 168/200
7020/7020 [==============================] - 0s 11us/step - loss: 12.8279 - acc: 0.2041
Epoch 169/200
7020/7020 [==============================] - 0s 11us/step - loss: 12.8279 - acc: 0.2041
Epoch 170/200
7020/7020 [==============================] - 0s 11us/step - loss: 12.8279 - acc: 0.2041
Epoch 171/200
7020/7020 [==============================] - 0s 11us/step - loss: 12.8279 - acc: 0

In [21]:
score

[12.86691668638137, 0.20170940287262626]